Make sure GPU is enabled
Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU

In [1]:
# Clone git repo
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 2332, done.
remote: Total 2332 (delta 0), reused 0 (delta 0), pack-reused 2332
Receiving objects: 100% (2332/2332), 360.24 MiB | 22.37 MiB/s, done.
Resolving deltas: 100% (1281/1281), done.


In [2]:
cd Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [1]:
# Install dependencies
!pip install -q -r requirements_gpu.txt
!apt-get install -qq libportaudio2

  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9jl50x5f/pillow-simd/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9jl50x5f/pillow-simd/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-bgdheizx
       cwd: /tmp/pip-install-9jl50x5f/pillow-simd/
  Complete output (174 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/PIL
  copying src/PIL/BdfFontFile.py -> build/lib.linux-x86_64-3.6/PIL
  copying src/PIL/BlpImagePlugin.py -> build/lib.linux-x86_64-3.6/PIL
  copying src/PIL/BmpImagePlugin.py -> build/lib.linux-x86_64-3.6/PIL
  copying src/PIL/BufrStubImagePlugin.py -> build/lib.linux-x86_64-3.6/PIL
  copying src/PIL/Containe

DEPRECATION: Could not build wheels for pillow-simd which do not use PEP 517. pip will fall back to legacy 'setup.py install' for these. pip 21.0 will remove support for this functionality. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9jl50x5f/pillow-simd/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9jl50x5f/pillow-simd/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-2ak73ubr/install-record.txt --single-version-externally-managed --user --prefix= --compile --install-headers /home/u5840417/.local/include/python3.6m/pillow-simd
         cwd: /tmp/pip-install-9jl50x5f/

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [4]:
# Download dataset
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:06, 54.9MB/s]
Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/pretrained.pt  


In [5]:
# Code for recording audio from the browser
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output


class InvokeButton(object):
    def __init__(self, title, callback):
        self._title = title
        self._callback = callback

    def _repr_html_(self):
        from google.colab import output
        callback_id = 'button-' + str(uuid.uuid4())
        output.register_callback(callback_id, self._callback)

        template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
            <script>
              document.querySelector("#{callback_id}").onclick = (e) => {{
                google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                e.preventDefault();
              }};
            </script>"""
        html = template.format(title=self._title, callback_id=callback_id)
        return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    b = b64decode(s.split(',')[1])
    with open('audio.wav','wb+') as f:
        f.write(b)
    return 'audio.wav'

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("../models/encoder/saved_models/ge2e_pretrained.pt")
vocoder_weights = Path("../models/vocoder/saved_models/wavernn/wavernn_pretrained.pt")
syn_dir = Path("../models/synthesizer/saved_models/logs-syne/checkpoints")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)


Loaded encoder "ge2e_pretrained.pt" trained to step 1564501
Found synthesizer "syne" trained to step 21000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at ../models/vocoder/saved_models/wavernn/wavernn_pretrained.pt


In [7]:
def testFunc(text="FAQ"):
    print(text)
    print("HOLE")

#@title Deep vocoder
def synth():
    text = "This is being said in my own voice.  The computer has learned to do an impression of me." #@param {type:"string"}
    print("Now recording for 10 seconds, say what you will...")
    record(10)
    print("Audio recording complete")
    in_fpath = Path("audio.wav")
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed = encoder.embed_utterance(preprocessed_wav)
    print("Synthesizing new audio...")
    with io.capture_output() as captured:
        specs = synthesizer.synthesize_spectrograms([text], [embed])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
InvokeButton('Start recording', testFunc)